# Questão 1

Você deve criar a seguinte CNN usando o Keras:
- Parte convolucional da rede MobileNetV2; 
    - Tal modelo deve se configurado para receber como entrada imagens RGB com dimensões espaciais 128x128
    - Todos os parâmetros (pesos e bias) dessa parte do modelo devem ser "congelados" (impedidos de atualizar);
    - Os parâmetros dessa parte do modelo devem ser obtidos do modelo pré-treinado na ImageNet .
- Uma camada densa com ativação softmax.

E configurar o seu processo de otimização para usar:
- Otimizador Adam;
- Cross-entropy como loss;
- Acurácia como métrica de sucesso.

Dicas:
1. Use uma camada do tipo Flatten para transformar o output da parte
convolucional do modelo em um vetor;
2. Use o parâmetro input_shape para configurar o tamanho da imagem de
input;
3. Use a propriedade trainable das camadas convolucionais para que elas
sejam "congeladas";

In [9]:
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [10]:
mobilenet = MobileNetV2(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet',
)

mobilenet.trainable = False

In [11]:
output_newmobilenet = mobilenet.output
output_newmobilenet = Flatten()(output_newmobilenet)
output_newmobilenet = Dense(units=7, activation='softmax')(output_newmobilenet)

mobilenet = Model(
    inputs=mobilenet.input,
    outputs=output_newmobilenet
)

mobilenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [12]:
mobilenet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Questão 2

Treine o modelo usando o seguinte protocolo:
- Permita que os dados de treinamento (somente os dados de treinamento) sejam distorcidos usando as seguintes transformações:
    - horizontal_flip=True;
    - zoom_range=0.2;
    - shear_range=10;
    - height_shift_range=0.1;
    - width_shift_range=0.1.
- Usando mini-batch de 32 imagens;
- O modelo deve ser treinado por 10 épocas (ou seja, que ele deve processar durante o treinamento uma quantidade igual a 10x o tamanho da base de treinamento).

Acompanhe a acurácia do modelo nos dados de treinamento e na base de teste ao
final de cada época.

Dicas:
1. Aplique a função de pré-processamento usada pela MobileNetV2;
2. Use dois ImageDataGenerator diferentes para que somente a base de treinamento tenha data augmentation;

In [13]:
batch_size = 32

In [14]:
train_image_data_gen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=lambda x : image.array_to_img(preprocess_input(image.img_to_array(x)))
)

train_gen = train_image_data_gen.flow_from_directory(
    directory='/content/drive/MyDrive/ICMC/Redes Neurais/Trabalho 3/dataset/train',
    target_size=(128,128),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 2284 images belonging to 7 classes.


In [15]:
test_image_data_gen = ImageDataGenerator(
    preprocessing_function=lambda x : image.array_to_img(preprocess_input(image.img_to_array(x)))
)

test_gen = test_image_data_gen.flow_from_directory(
    directory='/content/drive/MyDrive/ICMC/Redes Neurais/Trabalho 3/dataset/test',
    target_size=(128,128),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 1400 images belonging to 7 classes.


In [16]:
mobilenet.fit(
    x=train_gen,
    epochs=10,
    validation_data=test_gen,
)

Epoch 1/10
72/72 [==============================] - 30s 312ms/step - loss: 3.9901 - accuracy: 0.3147 - val_loss: 1.8178 - val_accuracy: 0.5279
Epoch 2/10
72/72 [==============================] - 21s 289ms/step - loss: 1.2299 - accuracy: 0.6115 - val_loss: 1.0843 - val_accuracy: 0.6607
Epoch 3/10
72/72 [==============================] - 20s 279ms/step - loss: 1.0273 - accuracy: 0.6636 - val_loss: 1.1570 - val_accuracy: 0.6564
Epoch 4/10
72/72 [==============================] - 19s 267ms/step - loss: 0.9267 - accuracy: 0.7049 - val_loss: 1.0255 - val_accuracy: 0.7079
Epoch 5/10
72/72 [==============================] - 21s 288ms/step - loss: 0.9192 - accuracy: 0.7027 - val_loss: 1.5293 - val_accuracy: 0.6293
Epoch 6/10
72/72 [==============================] - 21s 287ms/step - loss: 0.8791 - accuracy: 0.7145 - val_loss: 1.0984 - val_accuracy: 0.6893
Epoch 7/10
72/72 [==============================] - 21s 286ms/step - loss: 0.7478 - accuracy: 0.7581 - val_loss: 1.4364 - val_accuracy: 0.6336